## Word2Vec in keras. 
* based partially on: https://adventuresinmachinelearning.com/word2vec-keras-tutorial/

In [1]:
# We'll implement the skip-gram model first (CBOW later)

Note that to implement negative sampling, we'll follow the following procedure:
* Construct a binary output layer instead of multi-class
* Output is True if we are suppling the target word and a true context word
* Output is False if we are suppling the target word and a a negative sample (i.e. a word sampled which is NOT in the context of the target)
* To ensure that similar words end up having similar embeddings, we'll use cosine similarity. This should give a 1 when two words are in the same context and 0 otherwise
Broadly, the network will look like:
* An integer input (i.e. the index of the target word in the vocabulary) and a true or false context word
* An embedding layer lookup (i.e. looking up the embedding vector in the embedding matrix)
* The dot product operation (i.e. to compute the cosine similarity)
* The output layer: a sigmoid here.

In [192]:
# Import modules:
from __future__ import division

import os
import sys
import pandas as pd
import keras
import keras.backend as K
import numpy as np
import functools
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from IPython.display import SVG
from keras_sequential_ascii import sequential_model_to_ascii_printout
from keras.callbacks import History 
import urllib
import zipfile
import tensorflow as tf
import collections
sys.path.insert(0, '/Users/danielokeeffe/Documents/src/nlp_preprocessing/')
import nlp_preprocessing

In [193]:
# Reload nlp_preprocessing if you add any functions to it
reload(nlp_preprocessing)

<module 'nlp_preprocessing' from '/Users/danielokeeffe/Documents/src/nlp_preprocessing/nlp_preprocessing.py'>

In [311]:
class SimilarityCallback:
    def __init__(self, valid_size, dictionary, valid_examples, vocab_size, validation_model):
        self.valid_size = valid_size
        self.dictionary = dictionary
        self.valid_examples = valid_examples
        self.vocab_size = vocab_size
        self.validation_model = validation_model
        
    def run_sim(self):
        for i in range(self.valid_size):
            valid_word = self.dictionary[self.valid_examples[i]]
            # Define the number of nearest neighbours
            top_k = 8
            sim = self._get_sim(self.valid_examples[i])
            nearest = (-sim).argsort()[1:top_k+1]
            log_str = 'Nearest to {0}: '.format(valid_word)
            for k in range(top_k):
                close_word = self.dictionary[nearest[k]]
                log_str = '{0} {1},'.format(log_str, close_word)
            print(log_str)
            

    def _get_sim(self, valid_word_idx):
        sim = np.zeros((self.vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        for i in range(self.vocab_size):
            in_arr1[0,] = valid_word_idx
            in_arr2[0,] = i
            out = self.validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim

In [288]:
def plot_model(model):
    return SVG(keras.utils.vis_utils.model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [373]:
def predict_on_validation_model(word_list, dictionary, reversed_dictionary, validation_model, vocab_size, top_k=8):
    """A function to return the top_k most similar words after training of the primary model"""
    # Verify that the words in the input word list are actually in the dictionary
    try:
        word_indicies = [reversed_dictionary[word] for word in word_list]
    except KeyError:
        print('a supplied word is not in dictionary')
        return None
    for i in range(len(word_indices)):
        test_word = word_list[i]
        sim = validation_get_sim(valid_word_idx=word_indices[i], 
                                 validation_model=validation_model, 
                                 vocab_size=vocab_size)
        nearest = (-sim).argsort()[1:top_k+1]
        log_str = 'Nearest to {0}: '.format(test_word)
        for k in range(top_k):
            close_word = dictionary[nearest[k]]
            log_str = '{0} {1},'.format(log_str, close_word)
        print(log_str)

In [374]:
def validation_get_sim(valid_word_idx, validation_model, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    for i in range(vocab_size):
        in_arr1[0,] = valid_word_idx
        in_arr2[0,] = i
        out = validation_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

In [194]:
# Load data.  We'll use the text data at: 'http://mattmahoney.net/dc/' The filename is text8.zip
url = 'http://mattmahoney.net/dc/'
filename = download_file('text8.zip', url)

In [195]:
filename = nlp_preprocessing.download_file('text8.zip', url)

In [196]:
vocabulary = nlp_preprocessing.read_data_from_zip(filename=filename)

In [197]:
len(vocabulary)

1

In [198]:
len(vocabulary[0])

17005207

In [199]:
# This is pretty big, so we'll produce a trimmed version of the vocabulary of the n most common words, say 10,000
vocab_size = 10000

In [200]:
trimmed_vocabulary = nlp_preprocessing.trim_vocabulary(vocab=vocabulary, vocab_size=vocab_size)

In [201]:
len(trimmed_vocabulary)

1

In [202]:
len(trimmed_vocabulary[0])

10000

In [203]:
trimmed_vocabulary[0][0:10]

['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two']

In [204]:
# This is pretty big.  We'll use it to build the dictionary and reversed dictionary, but then remove it to save memory.
dictionary = nlp_preprocessing.create_dictionary(trimmed_vocabulary)

In [205]:
# Make the reversed dictionary
reversed_dictionary = nlp_preprocessing.create_reversed_dictionary(dictionary_=dictionary)

Gensim Dictionary type detected


In [206]:
len(dictionary)

10000

In [207]:
dictionary[1000]

u'bertrand'

In [208]:
len(reversed_dictionary)

10000

In [209]:
reversed_dictionary['bertrand']

1000

In [210]:
# So now we don't need the full vocabulary anymore.
del vocabulary

# Define some basic parameters

In [375]:
# the window of words around the target to will be used to draw context samples:
window_size = 3
# The size of each word embedding vector:
vector_dim = 300
# The number of epochs to train for. This is going to be big.  Even with negative sampling, creating a decent embedding matrix can take awhile
epochs = 50000

In [376]:
# We'll need to setup a validation set. This will be used to check to see what words grow in similarity as we train the network.
# We'll select a random set of words to measure similarity on
valid_size = 16
# We want to select these words from the top 100 most common words in the dataset.  We didn't explicitly create the dictionary this way...we'll do it at random for now...
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

# Implement skipgram

In [377]:
# The sampling table will make sure we sample both true and false context words in a balanced way and not just select the most common words
sampling_table = keras.preprocessing.sequence.make_sampling_table(vocab_size)
# We need the keys of the dictionary, i.e. the index of each word in the dictionary for the skipgram sampling. 
couples, labels = keras.preprocessing.sequence.skipgrams(dictionary.keys(), vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype='int32')
word_context = np.array(word_context, dtype='int32')

In [378]:
print(couples[:10], labels[:10])

([[8613, 9158], [5127, 4965], [8980, 2102], [8454, 9272], [2828, 9137], [9921, 3295], [2586, 2784], [5748, 8208], [5840, 7968], [5590, 9087]], [1, 1, 1, 1, 1, 1, 0, 1, 1, 0])


In [379]:
dictionary[7738]

u'renamed'

In [380]:
dictionary[1218]

u'broadcaster'

In [381]:
# To actually implement the embedding layers, we'll need to use the functional API.  We need to share a single embedding layer between two tensors (target word and context 
# words), and an additional output to measure similarity.

In [382]:
# Create input variables, one for the target, one for the context. We're just going to supply the target word and a context word individually, so the size of the inputs are both 1.
input_target = keras.Input((1,))
input_context = keras.Input((1,))

In [383]:
# Create the embedding layer (we should try out different initializers for the weights also):
embedding = keras.layers.Embedding(vocab_size, vector_dim, input_length=1, name='embedding')

In [384]:
# Now we need to be able to lookup target and context words in the embedding layer to measure similarity.
target = embedding(input_target)
target = keras.layers.Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = keras.layers.Reshape((vector_dim, 1))(context)

In [385]:
# Setup the similarity measurement
similarity = keras.layers.dot([target, context], normalize=True, axes=0)

In [386]:
dot_product = keras.layers.dot([target, context], normalize=False, axes=0)
dot_product = keras.layers.Reshape((1,))(dot_product)
output = keras.layers.Dense(1, activation='sigmoid')(dot_product)

In [387]:
# Compile the model
model = keras.Model(inputs=[input_target, input_context], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [388]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 300)       3000000     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
reshape_20 (Reshape)            (None, 300, 1)       0           embedding[0][0]                  
__________

In [389]:
sequential_model_to_ascii_printout(model)

           OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

               Input   #####           1
          InputLayer     |   -------------------         0     0.0%
                       #####           1
          InputLayer     |   -------------------         0     0.0%
                       #####           1
           Embedding   emb | -------------------   3000000    99.0%
                       #####      1  300
             Reshape     |   -------------------         0     0.0%
                       #####    300    1
             Reshape     |   -------------------         0     0.0%
                       #####    300    1
                 Dot   ????? -------------------         0     0.0%
                       #####      1    1
             Reshape     |   -------------------         0     0.0%
                       #####           1
               Dense   XXXXX -------------------         2     0.0%
             sigmoid   #####           1


In [390]:
# A nicer visualization of the word2vec skipgram model used here.
plot_model(model)

In [391]:
# To get to the validation we want to track, we'll initialize a secondary model. This model should share the the embedding layer with the primary model.  We aren't actually going to
# train this model, so we don't need to compile it.
validation_model = keras.Model(inputs=[input_target, input_context], outputs=similarity)

In [392]:
# Setup the similarity callback objecct
sim_cb = SimilarityCallback(valid_size=valid_size, 
                            dictionary=dictionary, 
                            valid_examples=valid_examples, 
                            vocab_size=vocab_size, 
                            validation_model=validation_model)

In [393]:
# Now we can train this thing.
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 1000 == 0:
        print('Iteration {0}, loss = {1}'.format(cnt, loss))
    if cnt % 10000 == 0:
        sim_cb.run_sim()

Iteration 0, loss = 0.69817429781
Nearest to aboard:  font, way, being, vector, reviewed, hoped, notably, demon,
Nearest to absorption:  editions, opinions, serves, defined, reportedly, unchanged, create, jonah,
Nearest to abbot:  caliph, surnames, becomes, decorated, mate, encryption, thousand, even,
Nearest to accidental:  colspan, filmed, counted, projected, paid, romans, momentum, bandwidth,
Nearest to acclaimed:  hughes, lead, jain, august, observances, mouth, aka, recommendations,
Nearest to abraham:  monastery, philosophers, albert, package, libertarian, gave, turn, defines,
Nearest to ability:  bassist, several, mathematical, completed, dublin, causes, saudi, goat,
Nearest to acid:  flesh, speculated, indonesian, fugue, false, seldom, range, hanover,
Nearest to accepted:  malaria, ammunition, northeast, encountered, basketball, disorder, asteroids, chemical,
Nearest to accused:  abu, empires, missouri, nights, county, love, neil, farms,
Nearest to achilles:  location, kingdoms,

In [372]:
predict_on_validation_model(word_list=['six', 'eight'], dictionary=dictionary, reversed_dictionary=reversed_dictionary, validation_model=validation_model, vocab_size=vocab_size)

Nearest to six:  stations, impulse, debut, ivoire, burst, bang, bodily, moore,
Nearest to eight:  intimate, augustine, inserted, genealogy, wax, winds, kennedy, favoured,
